# Laboratorio 3: Análisis de Asociación

UCR | ECCI | Análisis de Grandes Volúmenes de Datos - CI-0163 | I-2023  
Fabián Orozco Chaves - B95690  
Daniel Escobar Giraldo - C02748  


#  Ejercicio Práctico 1


## Dataset transaccional (9697 filas)

Para este laboratorio utilizaremos el siguiente dataset extraído de kaggle:
[Sales Product Data](https://www.kaggle.com/datasets/knightbearr/sales-product-data) para aplicar el algoritmo apriori.

Este es un dataset transaccional que contiene datos sobre ventas de productos y servicios. El dataset contiene:

- ID de pedido: Un ID de pedido es el sistema numérico que Amazon utiliza exclusivamente para hacer un seguimiento de los pedidos. Cada pedido recibe su propio ID de pedido que no se duplicará.
- Producto: El producto que se ha vendido.
- Cantidad Pedida: La cantidad total de artículos pedidos.
- Precio por producto: El precio de cada producto.
- Fecha del Pedido: Es la fecha en la que el cliente solicita el envío del pedido.
- Dirección de Compra: La orden de compra es preparada por el comprador, a menudo a través de un departamento de compras.

## Algoritmo Apriori





In [ ]:
import pandas as pd
from pandas import read_csv

url = 'https://raw.githubusercontent.com/DanielEscobar19/LaboratoriosBigData/main/Laboratorio3/Sales_January_2019.csv'

# leemos el dataset
df = pd.read_csv("Sales_January_2019.csv")

df.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,9697,9697,9697,9697,9697,9697
unique,9269,20,8,19,8077,9161
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,16,1171,8795,1171,16,16


---

Implementación del algoritmo apriori para el dataset cargado:

In [ ]:
# Agrupa los productos según su transacción (Order ID).
transacciones = df.groupby('Order ID')['Product'].apply(list).tolist()
transacciones = df.groupby('Order ID')['Product'].apply(list).reset_index().values.tolist()
transacciones = [[order_id, products] for order_id, products in transacciones]


print(f"productos agrupados:\n{transacciones}")
print(f"cantidad de grupos: {len(transacciones)}")

productos agrupados:
[['141234', ['iPhone']], ['141235', ['Lightning Charging Cable']], ['141236', ['Wired Headphones']], ['141237', ['27in FHD Monitor']], ['141238', ['Wired Headphones']], ['141239', ['AAA Batteries (4-pack)']], ['141240', ['27in 4K Gaming Monitor']], ['141241', ['USB-C Charging Cable']], ['141242', ['Bose SoundSport Headphones']], ['141243', ['Apple Airpods Headphones']], ['141244', ['Apple Airpods Headphones']], ['141245', ['Macbook Pro Laptop']], ['141246', ['AAA Batteries (4-pack)']], ['141247', ['27in FHD Monitor']], ['141248', ['Flatscreen TV']], ['141249', ['27in FHD Monitor']], ['141250', ['Vareebadd Phone']], ['141251', ['Apple Airpods Headphones']], ['141252', ['USB-C Charging Cable']], ['141253', ['AA Batteries (4-pack)']], ['141254', ['AAA Batteries (4-pack)']], ['141255', ['USB-C Charging Cable']], ['141256', ['Google Phone']], ['141257', ['Apple Airpods Headphones']], ['141258', ['AA Batteries (4-pack)']], ['141259', ['AAA Batteries (4-pack)']], ['141260

In [ ]:
import pandas as pd
from collections import Counter

# generamos una lista con los valores unique de los productos
itemset_1 = []
for transaccionActual in transacciones:
    for producto in transaccionActual[1]:
        if(producto not in itemset_1):
            itemset_1.append(producto)

# sorteamos el itemset de productos unicos
itemset_1 = sorted(itemset_1)

# soporte minimo que debe cumplir cada itemset
minSupport = 2
supportThreshold = minSupport/len(itemset_1)

# se cuentan las apariciones de cada valor del itemset
candidatos = Counter()
for producto in itemset_1:
    for transaccion in transacciones:
        if(producto in transaccion[1]):
            candidatos[producto] += 1

# mostramos la primera lista de candidatos
print("candidatos_1: ")
for i in candidatos:
    print(str([i])+": "+str(candidatos[i]))
print()

candidatosPodados = Counter()
for producto_soporte in candidatos:
    if(candidatos[producto_soporte] >= minSupport):
        candidatosPodados[frozenset([producto_soporte])]+=candidatos[producto_soporte]

# mostramos los candidatos podados
print("candidatos sin podados:")
for i in candidatosPodados:
    print(str(list(i))+": "+str(candidatosPodados[i]))
print()

# itemsets frecuenntes previos
itemsetsPrevios = candidatosPodados
podadasCantidad = 1

iteracion = 2
while True:
    # itemset por el que va la iteracion
    itemsetActual = set()

    # variable temporal para crear las combinaciones
    temporal = list(candidatosPodados)

    # creamos los subconjuntos no vacios de cada itemset
    for indexCandidato in range(0, len(temporal)):
        for indexCandidatoSiguiente in range(indexCandidato+1, len(temporal)):

            # recorre los candidatos y genera grupos del tamaño de la iteración
            unionTemporal = temporal[indexCandidato].union(temporal[indexCandidatoSiguiente])

            # agrega la union al itemset actual si es del tamaño de la iteracion
            if(len(unionTemporal) == iteracion):
                itemsetActual.add(unionTemporal)

    # convertimos a lista para poder iterar
    itemsetActual = list(itemsetActual)

    # candidatos actuales
    candidatos = Counter()

    # se cuentan las apariciones de cada valor del itemset
    for productos in itemsetActual:
        candidatos[productos] = 0
        for transaccion in transacciones:
            temporal = set(transaccion[1])
            if(productos.issubset(temporal)):
                candidatos[productos] += 1

    # mostramos la lista de candidatos actual
    print(f"Candidatos {str(iteracion)}:")
    for candidato in candidatos:
        # imprimimos el valor y la llave del diccionario canditatos
        print(f"{str(list(candidato))}: {str(candidatos[candidato])}")
    print()

    # candidatos luego de podar
    candidatosPodados = Counter()

    for candidato in candidatos:
        # la condicion poda los que no cumplen el soporte
        if(candidatos[candidato] >= minSupport):
            candidatosPodados[candidato] += candidatos[candidato]

    # imprimimos la lista de candidatos luego de ser podado
    print(f"Candidatos sin podados {str(iteracion)}:")
    for candidato in candidatosPodados:
        print(f"{str(list(candidato))} : {str(candidatosPodados[candidato])}")
    print()

    # si le itemset luego de podar esta vacio, detenemos el proceso
    if(len(candidatosPodados) == 0):
        break

    # asignamos el itemset previo para no perder el itemset final
    itemsetsPrevios = candidatosPodados
    podadasCantidad = iteracion

    # aumentamos las iteraciones para crear subsets mas grandes en la siguiente iteracion
    iteracion += 1

print(f"Resultado:\nCandidatos {str(podadasCantidad)}:")
for i in itemsetsPrevios:
    print(str(list(i))+": "+str(itemsetsPrevios[i]))
print()

candidatos_1: 
['20in Monitor']: 213
['27in 4K Gaming Monitor']: 312
['27in FHD Monitor']: 418
['34in Ultrawide Monitor']: 313
['AA Batteries (4-pack)']: 1038
['AAA Batteries (4-pack)']: 1082
['Apple Airpods Headphones']: 808
['Bose SoundSport Headphones']: 656
['Flatscreen TV']: 243
['Google Phone']: 316
['LG Dryer']: 39
['LG Washing Machine']: 42
['Lightning Charging Cable']: 1066
['Macbook Pro Laptop']: 235
['Product']: 1
['ThinkPad Laptop']: 216
['USB-C Charging Cable']: 1168
['Vareebadd Phone']: 124
['Wired Headphones']: 1003
['iPhone']: 379

candidatos sin podados:
['20in Monitor']: 213
['27in 4K Gaming Monitor']: 312
['27in FHD Monitor']: 418
['34in Ultrawide Monitor']: 313
['AA Batteries (4-pack)']: 1038
['AAA Batteries (4-pack)']: 1082
['Apple Airpods Headphones']: 808
['Bose SoundSport Headphones']: 656
['Flatscreen TV']: 243
['Google Phone']: 316
['LG Dryer']: 39
['LG Washing Machine']: 42
['Lightning Charging Cable']: 1066
['Macbook Pro Laptop']: 235
['ThinkPad Laptop']: 21

Reglas de asociacion. Para cambiar el minimo de confianza se debe cambiar el valor de la variable confianzaMinima.

In [ ]:
from itertools import combinations

######################### confianza minima para reglas de asociacion
confianzaMinima = 60;

print(f"Resultado reglas:")

def eliminarVacios(tuples):
    tuples = filter(None, tuples)
    return list(tuples)


# combinationsList contiene la lista de combinaciones del itemset actual
combinationsList = list()
for itemset in itemsetsPrevios:
    # creamos las combinaciones del itemset actual
    for i in range(len(itemset)):
        combinationsList += list(combinations(itemset, i))

    # combinationsList contiene la lista de combinaciones del itemset actual
    # eliminamos los conjuntos vacios 
    combinationsList = eliminarVacios(combinationsList)

    # para revisar lista de combinaciones actual
    # print(f"combinationsList {combinationsList}\n")

    # se cuentan las apariciones de cada valor del itemset
    # esto calcula los soportes
    soportes = Counter()
    for combinacion in combinationsList:
        for transaccion in transacciones:
            if(set(combinacion).issubset(set(transaccion[1]))):
                soportes[tuple(combinacion)] += 1

    # lista donde almacenar las reglas que cumplen un minimo de confianza
    reglasAsociacionFiltradas = list()

    for i in soportes:
        reglasAsociacionFiltradas.append(f'\nitemsetActual = {list(itemset)}')

        # formula vista en clase para las reglas de asociacion
        confianza = itemsetsPrevios[itemset]/soportes[i] * 100

        # string con regla de asociacion y su porcentaje
        regla = f'{list(i)} => {set(itemset)- set(i)}: supp({list(itemset)})/supp({str(list(i))}) == {itemsetsPrevios[itemset]}/{soportes[i]} = {confianza}%'

        # guardamos solo las que cumple con la confianza minima
        if (confianza >= confianzaMinima):
            reglasAsociacionFiltradas.append(f'{regla} Si cumple con la confianza minima ({confianzaMinima}%)')
        else:
            reglasAsociacionFiltradas.append(f'{regla} No cumple con la confianza minima ({confianzaMinima}%)')
    
    # print de los resultados ya filtrados
    for i in reglasAsociacionFiltradas:
        print(f"{i}")
    combinationsList.clear()


Resultado reglas:

itemsetActual = ['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones']
['Bose SoundSport Headphones'] => {'Google Phone', 'Wired Headphones'}: supp(['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones'])/supp(['Bose SoundSport Headphones']) == 2/656 = 0.3048780487804878% No cumple con la confianza minima (60%)

itemsetActual = ['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones']
['Google Phone'] => {'Bose SoundSport Headphones', 'Wired Headphones'}: supp(['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones'])/supp(['Google Phone']) == 2/316 = 0.6329113924050633% No cumple con la confianza minima (60%)

itemsetActual = ['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones']
['Wired Headphones'] => {'Bose SoundSport Headphones', 'Google Phone'}: supp(['Bose SoundSport Headphones', 'Google Phone', 'Wired Headphones'])/supp(['Wired Headphones']) == 2/1003 = 0.19940179461615154% No cumple con la confianza minima

## (Extra) Formas de mejorar la eficiencia del algoritmo  

1. Poda previa: Creemos que el algoritmo se puede optimizar si en cada iteración, en lugar de generar todos los posibles subconjuntos candidatos, genere solo los que cumplan el umbral de soporte mínimo (leímos un poco y sabemos que hay técnicas que podan estos conjuntos), esto haría que se reduzca el espacio de almacenamiento necesario y que reduzca la cantidad de subconjuntos en la siguiente iteración.

2. Estructuras optimizadas: para calcular el soporte de cada itemset hay que contar, se podrían usar estructuras como árboles o tablas hash para almacenar esos itemsets y que el conteo sea más rápido.

3. Paralelización: esto funcionaría si es un dataset demasiado grande y se tiene de la capacidad en hardware para hacerlo, en esta estrategia se distribuiría la carga de trabajo, cada "worker" en la misma iteración puede realizar la generación de los siguientes subconjuntos y podarlo de un bloque de trabajo.

#  Ejercicio Práctico 2




## Tutorial
El análisis de asociación trata de encontrar patrones comunes de elementos en grandes conjuntos de datos. Una aplicación específica suele denominarse análisis de la cesta de la compra. Esta busca ver la relación entre transacciones para descubrir que productos suelen comprarse juntos. Aunque este tipo de asociaciones se utilizan normalmente para analizar transacciones de ventas, el análisis básico puede aplicarse a otras situaciones, como el seguimiento de clics, los pedidos de piezas de recambio o los motores de recomendación en línea.

Veremos un ejemplo de uso de biblioteca mlxtend que ofree el algoitmo Apriori para analizar un conjunto relativamente grande de datos de venta online e intentar encontrar combinaciones de compra interesantes. 

El análisis de asociación es relativamente sencillo en cuanto a conceptos matemáticos y fácil de explicar a personas sin conocimientos técnicos. Además, es una herramienta de aprendizaje no supervisado que busca patrones ocultos, por lo que la necesidad de preparación de datos e ingeniería de características es limitada. 

Tutorial extraido de: [Practical Business Python](https://pbpython.com/market-basket-analysis.html)


###  Chris Moffitt - Introduction to Market Basket Analysis in Python

#### Términos básicos

Las reglas de asociación se escriben así {Pañales} -> {Cerveza} que significa que existe una fuerte relación entre los clientes que compraron pañales y también compraron cerveza en la misma transacción. {Pañales} es el antecedente y {Cerveza} es el consecuente. Tanto los antecedentes como los consecuentes pueden tener varios elementos.

El **soporte** es la frecuencia relativa con la que aparecen las reglas. En muchos casos, en general es bueno buscar un alto soporte para asegurarse de que se está usando relación útil.

La **confianza** es una medida de la fiabilidad de la regla. Si tenemos la relación {Diaper, Gum} -> {Beer, Chips}, una confianza de 0,5 en el significaría que en el 50% de los casos en los que se compraron pañales y chicles, también se compró cerveza y patatas fritas. 

**Lift** es la relación entre el apoyo observado y el esperado si las dos reglas fueran independiente. La regla es que un valor de Lift cercano a 1 significa que las reglas eran completamente independientes. Los valores > 1 suelen ser más "interesantes" y podrían indicar un patrón de reglas útil.

Procedemos al código:

#### Código del tutorial

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# cargamos el dataset
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
print(f'Dataset de prueba:\n {df.head()}')

# realizamos una liempieza de los datos eliminando los espacios en blanco en la descripcion
df['Description'] = df['Description'].str.strip()

# eliminamos las filas que no tienen un numero de identificacion (invoiceNo)
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# eliminamos las filas que contienen transacciones de credito (que tienen una C)
df = df[~df['InvoiceNo'].str.contains('C')]


# cambiamos las filas para que haya una transaccion por cada producto y si filtran productos solod e Francia para que hayan menos datos
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# convertimos todos los valores positivos a 1 y todos los valores negativos o iguales a 0 los reemplazamos con un 0
# esto completa el onehot encoding de los datos
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

# con las tranformaciones hechas se puede aplicar el algoritmo apriori
# se calculan los itemsets frecuentes. Podemos modificar el min_cupport para seleccionar mas rows
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

# finalmente generamos las reglas de asociacion
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

rules = rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

print(f'\nReglas filtradas:\n {rules}')

      

Dataset de prueba:
   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
Reglas filtradas:
                                           antecedents  \
2                        (ALARM CLOCK BAKELIKE GREEN)   
3                          (ALARM CL

In [ ]:

# sumando las apariciones del producto podemos ver cuantos de estos se vendieron en total
# sabiendo este dato se puede tomar decisiones para aumentar las ventas de un producto
x = basket['ALARM CLOCK BAKELIKE GREEN'].sum()

print(x)

x = basket['ALARM CLOCK BAKELIKE RED'].sum()

print(x)

340.0
316.0


In [ ]:

# podemos realizar el mismo proceso pero agarrando los productos de Alemania en lugar de Francia
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
6,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746


Parece ser que los alemanes compran mucho: Plasters in Tin Spaceboy y Woodland Animals.

### MLextend Apriori

#### Ejemplo 1 - Generando itemsets frecuentes

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# creamos un datset de prueba
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

# transforma el formato del dataset a una matriz de valores booleanos
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

# crea un dataframe donde cada columna es un elemento y cada fila es una transaccion
df = pd.DataFrame(te_ary, columns=te.columns_)

# aplicamos el algoritmo apriori con un soporte minimo del
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Yogurt, Kidney Beans)"


#### Ejemplo 2 - Selección y filtrado de resultados


In [ ]:

# guardamos en una variable el resultado del algoritmo apriori
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

# se agrega una columna de length al dataframe la cual muestra la longitud de cada itemset
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# filtramos los resultados para obtener solos los que tienen una logitud de 2 y un soporte mayor a 0.8
print("Filtro de longitud = 2 y sporte de 0.8:")
print(frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ])
print()
print("Filtro que bsuca solo los que tienen ajo y huevos:")
# seleccionamos los conjuntos iguales a ajo y huevos
print(frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ])



Filtro de longitud = 2 y sporte de 0.8:
   support              itemsets  length
5      0.8  (Eggs, Kidney Beans)       2

Filtro que bsuca solo los que tienen ajo y huevos:
   support       itemsets  length
6      0.6  (Onion, Eggs)       2


#### Ejemplo 3 - Trabajar con representaciones dispersas

In [ ]:
# se crea una matriz binaria y dispersa con transform
oht_ary = te.fit(dataset).transform(dataset, sparse=True)

# a partir de la representacion dispersa creamos dataframe
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

print("Matriz binaria: ")
print(sparse_df)

print()

print("Resultado de algoritmo apriori: ")
# aplicamos el algoritmo para comprobar que funciono la transformacion de datos
print(apriori(sparse_df, min_support=0.6, use_colnames=True))


Matriz binaria: 
   Apple  Corn  Dill  Eggs  Ice cream  Kidney Beans  Milk  Nutmeg  Onion  \
0      0     0     0     1          0          True     1       1      1   
1      0     0     1     1          0          True     0       1      1   
2      1     0     0     1          0          True     1       0      0   
3      0     1     0     0          0          True     1       0      0   
4      0     1     0     1          1          True     0       0      1   

   Unicorn  Yogurt  
0        0       1  
1        0       1  
2        0       0  
3        1       1  
4        0       0  

Resultado de algoritmo apriori: 
    support                     itemsets
0       0.8                       (Eggs)
1       1.0               (Kidney Beans)
2       0.6                       (Milk)
3       0.6                      (Onion)
4       0.6                     (Yogurt)
5       0.8         (Eggs, Kidney Beans)
6       0.6                (Onion, Eggs)
7       0.6         (Milk, Kidney Bean

## Conlusión

Con esto podemos ver varias formas de aplicar el algoritmo de apriori. En los ejemplos de la primera seccion Algoritmo Apriori realizamos el algoritmo manualmente utilizando pandas y python estándar. Los conjuntos los manejamos con dataframes de pandas y bucles y estructuras basicas de python.

En los ejemplos de los tutoriales utilizamos la librería mlxtend la cual transforma los datasets en una mtriz binaria para mostrar la presencia de elementos en cada transacción. Esto hace que la forma de calcular los conjuntos frecuentes sea diferente. La biblioteca aplica el algoritmo de apriori de una forma más eficiente y optimizada.